🌡️ Temperature Classification using DHT22
Classes: Cold – Warm – Hot
1️⃣ Problem Definition (Very Important)
Sensor

DHT22 temperature sensor

Output: Temperature in °C (continuous value)

Goal

Convert a continuous temperature into discrete classes:Cold

Warm

Hot

👉 This is a CLASSIFICATION problem, not regression.

2️⃣ Define the Classes (Domain Knowledge)

You must define ranges first (this is important for teaching).

Example class definition
Class	Temperature Range (°C)
Cold	< 20
Warm	20 – 30
Hot	> 30

📌 You can adjust these ranges based on location.

3️⃣ Dataset Creation (Simple & Realistic)
Example dataset format
Temperature (°C)	Label
16.5	Cold
22.0	Warm
28.5	Warm
32.0	Hot
35.5	Hot

Python label encoding

In [ ]:
def label_temperature_norm(temp_norm):
    temp = temp_norm * 50.0  # convert back to °C
    if temp < 20:
        return 0   # Cold
    elif temp <= 30:
        return 1   # Warm
    else:
        return 2   # Hot


Class mapping:

In [ ]:
class_names = ["Cold", "Warm", "Hot"]

Create Training Data (Synthetic or Real)

In [ ]:
import numpy as np
import sklearn

# Generate synthetic temperature data
temps = np.random.uniform(10, 40, 300)
temps_norm = temps / 50.0

labels = np.array([label_temperature_norm(t) for t in temps_norm])

X = temps_norm.reshape(-1, 1)
y = labels


In [ ]:
# Normalize (DHT22 max range ≈ 50°C)
X = X / 50.0

⚠️ Golden Rule:

Same normalization must be applied on STM32 / ESP32.
5️⃣ Split Data (Train / Test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

6️⃣ Build Embedded-Friendly Model

Minimal neural network (perfect for MCUs):

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 195 (780.00 B)

 Trainable params: 195 (780.00 B)

 Non-trainable params: 0 (0.00 B)

7️⃣ Train the Model

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.1,
    verbose=1
)

Epoch 1/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.3089 - loss: 1.0998 - val_accuracy: 0.3750 - val_loss: 1.0988
Epoch 2/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3180 - loss: 1.0992 - val_accuracy: 0.3333 - val_loss: 1.0984
Epoch 3/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3720 - loss: 1.0983 - val_accuracy: 0.3333 - val_loss: 1.0983
Epoch 4/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3327 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0984
Epoch 5/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3799 - loss: 1.0982 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 6/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3379 - loss: 1.0984 - val_accuracy: 0.3333 - val_loss: 1.0985
Epoch 7/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3312 - loss: 1.0984 - val_accuracy: 0.3333 - val_loss: 1.0983
Epoch 8/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3911 - loss: 1.0976 - val_accuracy: 0.3333 - 

8️⃣ Evaluate the Model

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7931 - loss: 0.7960
Test Accuracy: 0.7833333611488342


9️⃣ Test with Known Temperatures (Demo Cell)

In [ ]:
test_temps = np.array([-10, 25, 35]).reshape(-1, 1)
test_temps_norm = test_temps / 50.0

pred = model.predict(test_temps_norm)
pred_classes = np.argmax(pred, axis=1)

for t, c in zip(test_temps.flatten(), pred_classes):
    print(f"{t} °C → {class_names[c]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
-10 °C → Cold
25 °C → Hot
35 °C → Hot


In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{np.int64(0): np.int64(94),
 np.int64(1): np.int64(107),
 np.int64(2): np.int64(99)}

🔟 Confidence Interpretation (Very Important)

In [ ]:
confidence = np.max(pred, axis=1)

for t, conf in zip(test_temps.flatten(), confidence):
    print(f"Confidence: {conf:.2f}")


Confidence: 0.49
Confidence: 0.48
Confidence: 0.47


In [ ]:
pred = model.predict(test_temps_norm)
pred_class = np.argmax(pred, axis=1)
confidence = np.max(pred, axis=1)

for t, c, conf in zip(test_temps.flatten(), pred_class, confidence):
    if conf < 0.7:
        print(f"{t} °C → Uncertain")
    else:
        print(f"{t} °C → {class_names[c]} ({conf:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
15 °C → Uncertain
25 °C → Uncertain
35 °C → Uncertain


1️⃣1️⃣ Convert to TensorFlow Lite (For Embedded)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("dht22_temp_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model size:", len(tflite_model), "bytes")


Saved artifact at '/tmp/tmprcj_78lp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  135739290735760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135739290735376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135739290734032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135739290738640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135739290742096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135739290745168: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model size: 2900 bytes
